#**Lab 4 Assignment: Data Preparation with Pandas**

In [15]:
import pandas as pd
import requests

df=pd.read_csv('https://data.cityofnewyork.us/api/views/5rq2-4hqu/rows.csv?accessType=DOWNLOAD')

df.head(5)


interest_columns = ['spc_latin', 'spc_common', 'status', 'health', 'tree_dbh', 'stump_diam']
df_reduced = df[interest_columns]

df_reduced

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
0,Acer rubrum,red maple,Alive,Fair,3,0
1,Quercus palustris,pin oak,Alive,Fair,21,0
2,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,3,0
3,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,10,0
4,Tilia americana,American linden,Alive,Good,21,0
...,...,...,...,...,...,...
683783,Quercus palustris,pin oak,Alive,Good,25,0
683784,Cladrastis kentukea,Kentucky yellowwood,Alive,Good,7,0
683785,Acer rubrum,red maple,Alive,Good,12,0
683786,Acer rubrum,red maple,Alive,Good,9,0


In [16]:
# count nulls

nullValue_counts=df_reduced.isnull().sum()
nullValue_counts

spc_latin     31619
spc_common    31619
status            0
health        31616
tree_dbh          0
stump_diam        0
dtype: int64

In [17]:
# nan finding

NaNsOnly=df_reduced.loc[(df_reduced['spc_latin'].isnull() ) | (df_reduced['spc_common'].isnull() ) | (df_reduced['health'].isnull() ) ]
NaNsOnly.head()

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
61,NaN,NaN,Dead,NaN,2,0
307,NaN,NaN,Stump,NaN,0,9
370,NaN,NaN,Dead,NaN,4,0
494,NaN,NaN,Stump,NaN,0,17
556,NaN,NaN,Dead,NaN,2,0


In [18]:
pd.value_counts(NaNsOnly['status'])

status
Stump    17654
Dead     13961
Alive        6
Name: count, dtype: int64

In [19]:
# create a new df with species with at least one non-zero diameter

df_reduced_zerosFiltered =  df_reduced.loc[(df_reduced['tree_dbh'] != 0) | (df_reduced['stump_diam'] != 0)]

df_reduced_zerosFiltered
len(df_reduced_zerosFiltered)

683510

In [20]:
# group by criteria

group_criteria_columns = ['status', 'spc_latin', 'spc_common', 'health']

df_reduced_zerosFiltered_grouped = df_reduced_zerosFiltered.groupby(group_criteria_columns, dropna=False).mean("tree_dbh")

df_reduced_zerosFiltered_grouped

tree_dbh  \
status spc_latin                          spc_common    health              
Alive  Acer                               maple         Fair    11.903991   
                                                        Good    11.778865   
                                                        Poor    10.509434   
       Acer buergerianum                  trident maple Fair     9.750000   
                                                        Good     7.724138   
...                                                                   ...   
       NaN                                NaN           Good    13.000000   
                                                        Poor    11.000000   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN     12.000000   
       NaN                                NaN           NaN      5.492017   
Stump  NaN                                NaN           NaN      0.000000   

                                                                stump_diam  
status spc_latin                          spc_common    health              
Alive  Acer                               maple         Fair      0.000000  
                                                        Good      0.000000  
                                                        Poor      0.000000  
       Acer buergerianum                  trident maple Fair      0.000000  
                                                        Good      0.000000  
...                                                                    ...  
       NaN                                NaN           Good      0.000000  
                                                        Poor      0.000000  
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN       0.000000  
       NaN                                NaN           NaN       0.000000  
Stump  NaN                                NaN           NaN      16.750481  

[398 rows x 2 columns]

In [21]:
# index reset

df_reduced_zerosFiltered_grouped = df_reduced_zerosFiltered_grouped.reset_index()

df_reduced_zerosFiltered_grouped

,status,spc_latin,spc_common,health,tree_dbh,stump_diam
0,Alive,Acer,maple,Fair,11.903991,0.000000
1,Alive,Acer,maple,Good,11.778865,0.000000
2,Alive,Acer,maple,Poor,10.509434,0.000000
3,Alive,Acer buergerianum,trident maple,Fair,9.750000,0.000000
4,Alive,Acer buergerianum,trident maple,Good,7.724138,0.000000
...,...,...,...,...,...,...
393,Alive,NaN,NaN,Good,13.000000,0.000000
394,Alive,NaN,NaN,Poor,11.000000,0.000000
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,12.000000,0.000000
396,Dead,NaN,NaN,NaN,5.492017,0.000000


In [22]:
# create a diametre columns and remove others

df_reduced_zerosFiltered_grouped["diameter"] = df_reduced_zerosFiltered_grouped["tree_dbh"] + df_reduced_zerosFiltered_grouped["stump_diam"]

df_reduced_zerosFiltered_grouped.drop(columns=['tree_dbh', 'stump_diam'], inplace=True)
df_reduced_zerosFiltered_grouped


,status,spc_latin,spc_common,health,diameter
0,Alive,Acer,maple,Fair,11.903991
1,Alive,Acer,maple,Good,11.778865
2,Alive,Acer,maple,Poor,10.509434
3,Alive,Acer buergerianum,trident maple,Fair,9.750000
4,Alive,Acer buergerianum,trident maple,Good,7.724138
...,...,...,...,...,...
393,Alive,NaN,NaN,Good,13.000000
394,Alive,NaN,NaN,Poor,11.000000
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,12.000000
396,Dead,NaN,NaN,NaN,5.492017


In [23]:
df_reduced_counts=df_reduced.assign(count = 1)
df_reduced_counts.head()

,spc_latin,spc_common,status,health,tree_dbh,stump_diam,count
0,Acer rubrum,red maple,Alive,Fair,3,0,1
1,Quercus palustris,pin oak,Alive,Fair,21,0,1
2,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,3,0,1
3,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,10,0,1
4,Tilia americana,American linden,Alive,Good,21,0,1


In [24]:
# group_criteria_columns = ['status', 'spc_latin', 'spc_common', 'health'] # Still in the scope with needed data

df_reduced_counts_grouped = df_reduced_counts.groupby(group_criteria_columns, dropna=False).sum("count")

df_reduced_counts_grouped

tree_dbh  \
status spc_latin                          spc_common    health             
Alive  Acer                               maple         Fair       20582   
                                                        Good       55396   
                                                        Poor        6684   
       Acer buergerianum                  trident maple Fair         195   
                                                        Good         672   
...                                                                  ...   
       NaN                                NaN           Good          52   
                                                        Poor          11   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN           12   
       NaN                                NaN           NaN        76361   
Stump  NaN                                NaN           NaN            0   

                                                                stump_diam  \
status spc_latin                          spc_common    health               
Alive  Acer                               maple         Fair             0   
                                                        Good             0   
                                                        Poor             0   
       Acer buergerianum                  trident maple Fair             0   
                                                        Good             0   
...                                                                    ...   
       NaN                                NaN           Good             0   
                                                        Poor             0   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN              0   
       NaN                                NaN           NaN              0   
Stump  NaN                                NaN           NaN         295713   

                                                                count  
status spc_latin                          spc_common    health         
Alive  Acer                               maple         Fair     1733  
                                                        Good     4708  
                                                        Poor      639  
       Acer buergerianum                  trident maple Fair       20  
                                                        Good       87  
...                                                               ...  
       NaN                                NaN           Good        4  
                                                        Poor        1  
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN         1  
       NaN                                NaN           NaN     13960  
Stump  NaN                                NaN           NaN     17654  

[398 rows x 3 columns]

In [25]:
# Clean-up

df_reduced_counts_grouped.drop(columns=['tree_dbh','stump_diam'], inplace=True)
df_reduced_counts_grouped=df_reduced_counts_grouped.reset_index()
df_reduced_counts_grouped.head()

,status,spc_latin,spc_common,health,count
0,Alive,Acer,maple,Fair,1733
1,Alive,Acer,maple,Good,4708
2,Alive,Acer,maple,Poor,639
3,Alive,Acer buergerianum,trident maple,Fair,20
4,Alive,Acer buergerianum,trident maple,Good,87


In [26]:
# final df

final_df = df_reduced_counts_grouped.merge(df_reduced_zerosFiltered_grouped, how="left")

final_df=final_df.set_index(['status','spc_latin','spc_common','health'])
final_df


final_df.to_csv("final_df.csv")